In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from collections import Counter
from sklearn.model_selection import train_test_split

from konlpy.tag import Komoran

In [2]:
data = pd.read_csv('../data/total_data.csv', encoding='UTF8')

In [3]:
data.head()

,title,description,url,data_type,source,ori_label,ori_source,label
0,충청남도 당진시_산업단지 입주기업 현황_20220304,"충청남도 당진시 산업단지 입주 기업 현황입니다.(연번, 단지명, 회사명, 업종명, ...",https://www.data.go.kr/data/15106898/fileData.do,텍스트,충청남도 당진시,산업·통상·중소기업,공공데이터포털,제조소비
1,전라남도 강진군_미곡처리장현황_20221006,"전라남도 강진군 미곡처리장 현황에 대한 데이터로소 시군, 명칭, 소재지, 관리부서,...",https://www.data.go.kr/data/15106993/fileData.do,텍스트,전라남도 강진군,농림,공공데이터포털,식품건강
2,경상남도 거창군_월별 코로나 확진자 현황_20221007,경상남도 거창군 월별 코로나19 관련 확진자 및 사망자 수 현황 데이터로 월별 확진...,https://www.data.go.kr/data/15098865/fileData.do,텍스트,경상남도 거창군,보건,공공데이터포털,보건의료
3,충청남도 부여군_1인당지방세부담액_20221006,상기 데이터는 연도별 주민 1인당 또는 세대 당 부담된 지방세액에 대한 정보로 조...,https://www.data.go.kr/data/15080007/fileData.do,텍스트,충청남도 부여군,재정·세제·금융,공공데이터포털,경제금융
4,전라남도 보성군_담배소매인지정현황_20221006,"전라남도 보성군 담배소매인지정현황에 관한 공공데이터로 업소명, 지번주소, 도로명주소...",https://www.data.go.kr/data/15035564/fileData.do,텍스트,전라남도 보성군,일반공공행정,공공데이터포털,행정법률


In [4]:
data['description'].isnull().sum()

2

In [5]:
tokenizer = Komoran()

In [6]:
def preprocess(df):
    data['description'].dropna(axis=0)
    print("replace start..")
    for i in range(len(data)):
        data['description'][i] = str(data['description'][i]).replace('\n','').replace('\t','').replace('\r','').replace('.', '').replace(',','').replace("'","").replace('·', ' ').replace('=','').replace('·','')
    print("done..")
    
    print("tokenizing start..")
    description_tokenized = [[token+"/"+POS for token, POS in tokenizer.pos(des_)] for des_ in data['description']]
    exclusion_tags = ['JKS', 'JKC', 'JKG', 'JKO', 'JKB', 'JKV', 'JKQ', 'JX', 'JC',
                      'SF', 'SP', 'SS', 'SE', 'SO', 'EF', 'EP', 'EC', 'ETN', 'ETM',
                      'XSN', 'XSV', 'XSA']
    
    f = lambda x: x in exclusion_tags
    
    des_preprocessed = []
    for i in range(len(description_tokenized)):
        temp = []
        for j in range(len(description_tokenized[i])):
            if f(description_tokenized[i][j].split('/')[1]) is False:
                temp.append(description_tokenized[i][j].split('/')[0])
        des_preprocessed.append(temp)
    print("done..")
    
    words = np.concatenate(des_preprocessed).tolist()
    counter = Counter(words)
    counter = counter.most_common(20000-1)
    vocab = ['UNK'] + [key for key, _ in counter]
    word_to_index = {word:index for index, word in enumerate(vocab)}
    
    def wordlist_to_indexlist(wordlist):
        return [word_to_index[word] if word in word_to_index else word_to_index['UNK'] for word in wordlist]
    
    indexlist = list(map(wordlist_to_indexlist, des_preprocessed))
    
    return des_preprocessed, word_to_index, indexlist

In [8]:
# for i in range(len(data)):
#     try:
#         Komoran.pos(data['description'][i])
#     except:
#         print(f'{i} 번째 error발생')

In [10]:
des_result, word_to_index, index_list = preprocess(data)

replace start..
done..
tokenizing start..
done..


In [11]:
len(des_result)

127051

In [12]:
len(word_to_index)

20000

In [15]:
import pickle

In [ ]:
# with open("../data/des_embedding_bow.pkl","wb") as f:
#     pickle.dump(index_list, f)

In [16]:
with open("../data/des_embedding_bow.pkl","rb") as f:
    emb_load = pickle.load(f)

In [17]:
len(emb_load)

127051

## 랜덤 인덱스를 통해 유사한 데이터 뽑아내기

In [115]:
from numpy import dot
from numpy.linalg import norm

def cos_sim(A, B):
    return dot(A, B)/(norm(A)*norm(B))

In [116]:
import random
from random import randrange

random.seed(7777)
random_idx = randrange(len(data))

In [117]:
random_idx

107092

In [139]:
def print_data_info(random_idx):
    print("##########################################")
    print('data title : ' + str(data['title'][random_idx]))
    print('data description : ' + str(data['description'][random_idx]))
    print('data url : ' + str(data['url'][random_idx]))
    print('data type : ' + str(data['data_type'][random_idx]))
    print('data source : ' + str(data['source'][random_idx]))
    print('data ori_label : ' + str(data['ori_label'][random_idx]))
    print('data ori_source : ' + str(data['ori_source'][random_idx]))
    print('data label : ' + str(data['label'][random_idx]))
    print("##########################################")
    print("\n")

In [119]:
print_data_info(random_idx)

##########################################
data title : 대전광역시_중구_인테리어업체현황
data description : 대전광역시 중구 관내 인테리어업체 현황(연번 상호 업종 영업소재지도로명 전화번호 데이터기준일자) 을 제공합니다
data url : https://www.data.go.kr/data/15038457/fileData.do
data type : CSV
data source : 공공데이터포털
data ori_label : 행정법률
data ori_source : 빅데이터지도
data label : 행정법률
##########################################




In [120]:
target_des = data['description'][random_idx]

In [121]:
target_des

'대전광역시 중구 관내 인테리어업체 현황(연번 상호 업종 영업소재지도로명 전화번호 데이터기준일자) 을 제공합니다'

In [122]:
target_des = str(target_des).replace('\n','').replace('\t','').replace('\r','').replace('.', '').replace(',','').replace("'","").replace('·', ' ').replace('=','').replace('·','')
target_des = [token+"/"+POS for token, POS in tokenizer.pos(target_des)]

In [123]:
exclusion_tags = ['JKS', 'JKC', 'JKG', 'JKO', 'JKB', 'JKV', 'JKQ', 'JX', 'JC',
                      'SF', 'SP', 'SS', 'SE', 'SO', 'EF', 'EP', 'EC', 'ETN', 'ETM',
                      'XSN', 'XSV', 'XSA']
    
f = lambda x: x in exclusion_tags
    
target_preprocessed = []
for i in range(len(target_des)):
    if f(target_des[i].split('/')[1]) is False:
        target_preprocessed.append(target_des[i].split('/')[0])

In [124]:
target_preprocessed[:5]

['대전광역시', '중구', '관내', '인테리어', '업체']

In [125]:
 def wordlist_to_indexlist(wordlist):
        return [word_to_index[word] if word in word_to_index else word_to_index['UNK'] for word in wordlist]

In [126]:
target_index = wordlist_to_indexlist(target_preprocessed)

### 패딩 맞추기

In [127]:
max_len = max(len(item) for item in index_list)
max_len

470

In [128]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_index_list = pad_sequences(index_list, padding='post')

In [129]:
while len(target_index) < max_len:
    target_index.append(0)

In [130]:
len(target_index)

470

In [131]:
cal_cos_sim = []
for i, des in enumerate(padded_index_list):
    cal_cos_sim.append((cos_sim(target_index, des),i))

C:\Users\Home\AppData\Local\Temp/ipykernel_13788/3121412391.py:5: RuntimeWarning: invalid value encountered in true_divide
  return dot(A, B)/(norm(A)*norm(B))


In [132]:
cal_cos_sim.sort(reverse=True)

In [133]:
cal_cos_sim[:6]

[(1.0000000000000002, 5374),
 (0.9534720955310304, 94663),
 (0.9521772773968229, 8487),
 (0.950745265095663, 72540),
 (0.9423832589849174, 11397),
 (0.9420033858169019, 68898)]

In [134]:
result_index = [index[1] for index in cal_cos_sim[:6]]

In [135]:
result_index

[5374, 94663, 8487, 72540, 11397, 68898]

In [140]:
for index in result_index:
    print_data_info(index)

##########################################
data title : 대전광역시_중구_인테리어업체현황_20220902
data description : 대전광역시 중구 관내 인테리어업체 현황(연번 상호 업종 영업소재지도로명 전화번호 데이터기준일자) 을 제공합니다
data url : https://www.data.go.kr/data/15038457/fileData.do
data type : 텍스트
data source : 대전광역시 중구
data ori_label : 일반공공행정
data ori_source : 공공데이터포털
data label : 행정법률
##########################################


##########################################
data title : 서울특별시 은평구_가로등분전함 정보
data description : 은평구 가로등 분전함 설치현황입니다 속성은 구분 노선명 공동주택명 계약종별가로등분전함 위치로 구성되어있습니다
data url : https://www.data.go.kr/data/3078204/fileData.do
data type : NONE
data source : 공공데이터포털
data ori_label : 재난안전
data ori_source : 빅데이터지도
data label : 재난안전
##########################################


##########################################
data title : 경상남도 김해시_시민정보화교육_20220817
data description : 시민정보화교육(교육장 교육시작일 교육종료일 교육시작시간 교육종료시간 과목 대상 교육장주소 등)에 관한 데이터를 제공합니다
data url : https://www.data.go.kr/data/3063738/fileData.do
data type : 텍스트
data source : 경상